Default code from kaggle

In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Getting list of all possible outputs as keys and images of same class as values

In [41]:
import csv

data = {}
with open("../input/state-farm-distracted-driver-detection/driver_imgs_list.csv") as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        key = row[1].lower()
        if key in data:
            data[key].append(row[2])
        else:
            data[key] = [row[1]]

In [42]:
driver_actions_list = list(data.keys())
driver_actions_list

['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

Creating necessary folders

In [43]:
import os
import shutil

shutil.rmtree('master_data', ignore_errors=True)
shutil.rmtree('predict', ignore_errors=True)

os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

os.mkdir('predict')
os.mkdir('predict/all_class')

Creating necessary sub-folders

In [44]:
for action in driver_actions_list:
    os.mkdir(os.path.join('master_data/training',action))
    os.mkdir(os.path.join('master_data/testing',action))

Dividing data (80% for training and 20% for validation)

In [45]:
from shutil import copyfile
split_size = 0.8

for file in os.listdir('../input/state-farm-distracted-driver-detection/imgs/train'):
    file_path = '../input/state-farm-distracted-driver-detection/imgs/train'+'/{}'.format(file)
    images = os.listdir(file_path)
    train_size = int(split_size*len(images))
    train_images = images[:train_size]
    test_images = images[train_size:]
    
    for image in train_images:
        source = os.path.join(file_path,image)
        dest = os.path.join('./master_data/training',file,image)
        copyfile(source,dest)
    
    for image in test_images:
        source = os.path.join(file_path,image)
        dest = os.path.join('./master_data/testing',file,image)
        copyfile(source,dest)

Importing necessary Libraries

In [46]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

Creating CNN Model

In [47]:
model = tf.keras.models.Sequential([
    Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(128,128,3), kernel_initializer='glorot_normal'),
    MaxPooling2D(2,2),
    Conv2D(filters=128, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'),
    MaxPooling2D(2,2),
    Conv2D(filters=256, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'),
    MaxPooling2D(2,2),
    Conv2D(filters=512, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'),
    MaxPooling2D(2,2),
    Dropout(0.5),
    Flatten(),
    Dense(512,activation='relu', kernel_initializer='glorot_normal'),
    Dropout(0.5),
    Dense(10, activation='softmax', kernel_initializer='glorot_normal')
])
model.compile(optimizer = Adam(learning_rate = 0.01), loss = 'categorical_crossentropy',metrics= ['acc'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 128, 128, 64)      832       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 64, 64, 128)       32896     
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 32, 32, 256)       131328    
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 16, 16, 512)      

Using ImageDataGenerator for image processing

In [48]:
train_dir = './master_data/training'
test_dir = './master_data/testing'

train_datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (128,128),
    class_mode = 'categorical',
    batch_size = 128
)

test_datagen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (128,128),
    class_mode = 'categorical',
    batch_size = 128
)

Found 17934 images belonging to 10 classes.
Found 4490 images belonging to 10 classes.


EarlyStopping if model is not improving for 2 epochs continuously

In [49]:
es = EarlyStopping(monitor='val_acc',patience=4,min_delta=0.01)

Model Training

In [50]:
model.fit(
    train_generator,
    epochs=10,
    verbose=1,
    validation_data=test_generator,
    callbacks=[es],
    shuffle=True)

Epoch 1/10
141/141 [==============================] - 679s 5s/step - loss: 4.5685 - acc: 0.1685 - val_loss: 1.4595 - val_acc: 0.5203
Epoch 2/10
141/141 [==============================] - 681s 5s/step - loss: 1.3523 - acc: 0.5267 - val_loss: 0.7606 - val_acc: 0.7684
Epoch 3/10
141/141 [==============================] - 675s 5s/step - loss: 0.8950 - acc: 0.7044 - val_loss: 0.4625 - val_acc: 0.8873
Epoch 4/10
141/141 [==============================] - 672s 5s/step - loss: 0.7177 - acc: 0.7666 - val_loss: 0.2976 - val_acc: 0.9285
Epoch 5/10
141/141 [==============================] - 672s 5s/step - loss: 0.6039 - acc: 0.8080 - val_loss: 0.3206 - val_acc: 0.9134
Epoch 6/10
141/141 [==============================] - 674s 5s/step - loss: 0.5687 - acc: 0.8152 - val_loss: 0.2569 - val_acc: 0.9396
Epoch 7/10
141/141 [==============================] - 672s 5s/step - loss: 0.4988 - acc: 0.8410 - val_loss: 0.1858 - val_acc: 0.9572
Epoch 8/10
141/141 [==============================] - 675s 5s/step - 

Copying images to seprate folder for prediction

In [65]:
origin = '../input/state-farm-distracted-driver-detection/imgs/test/'
target = './predict/all_class/'

files = os.listdir(origin)

images_name = []
for file_name in files:
    shutil.copy(origin+file_name,target+file_name)
    images_name.append(file_name)

Using ImageDataGenerator for image processing

In [66]:
predict_dir = './predict/'

predict_datagen = ImageDataGenerator(rescale=1.0/255)
predict_generator = predict_datagen.flow_from_directory(
    predict_dir,
    target_size = (128,128),
    class_mode = None,
    batch_size = 128
)

Found 79726 images belonging to 1 classes.


Image prediction

In [69]:
filenames = predict_generator.filenames
nb_samples = len(filenames)

predict = model.predict(predict_generator,nb_samples)

In [ ]:
df = pd.DataFrame(predict, columns=['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9'])
df.insert(loc = 0,column='img',value=images_name)
df.head()

In [87]:
df.to_csv('submission.csv',index=False,header=True)